In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random 
import torch as T
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset, ConcatDataset, IterableDataset
import numpy as np
import pandas as pd

import math
import argparse
import import_ipynb
from brits2_i_original import *
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from argparse import ArgumentParser

save_path = "data/saved_models/activityReverse.tar"#vaegan_model - Copy.tar"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

if not os.path.exists("data/saved_models"):
    os.makedirs("data/saved_models")

importing Jupyter notebook from brits2_i_original.ipynb
importing Jupyter notebook from rits2_i_original.ipynb


In [2]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--nfeatures', default=609, type=int)
ARG_PARSER.add_argument('--dfeatures', default=43, type=int)
ARG_PARSER.add_argument('--ehidden', default=300, type=int)
ARG_PARSER.add_argument('--model', type=str)

ARG_PARSER.add_argument('--ehr', default=True)

ARG_PARSER.add_argument('--num_epochs', default=100, type=int)
ARG_PARSER.add_argument('--seq_len', default=20, type=int)
ARG_PARSER.add_argument('--pred_len', default=8, type=int)
ARG_PARSER.add_argument('--missingRate', default=10, type=int)
ARG_PARSER.add_argument('--patience', default=100, type=int)
ARG_PARSER.add_argument('--e_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--g_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--d_lrn_rate', default=0.001, type=float)
ARG_PARSER.add_argument('--resume_training', default=False)
ARG_PARSER.add_argument('--train', default=False)
ARG_PARSER.add_argument('--evalImp', default=True)
ARG_PARSER.add_argument('--evalPred', default=False)

_StoreAction(option_strings=['--eval'], dest='eval', nargs=None, const=None, default=True, type=None, choices=None, help=None, metavar=None)

In [3]:
ARGS = ARG_PARSER.parse_args(args=[])
MAX_SEQ_LEN = ARGS.seq_len
BATCH_SIZE = ARGS.batch_size
EPSILON = 1e-40

In [4]:
# Create Dataset
class CSVDataset(Dataset):
    def __init__(self, path, chunksize,length,seq_len,flag):
        self.path = path
        self.chunksize = chunksize
        self.len = int(length)#number of times total getitem is called
        self.seq_len=seq_len
        self.flag=flag
        self.reader=pd.read_csv(
                self.path,header=0,
                chunksize=self.chunksize)#,names=['data']))

    def __getitem__(self, index):
        data = self.reader.get_chunk(self.chunksize)
#         sex=pd.read_csv('C:\\Users/mehak/Desktop/demo.csv',header=0)
#         sex=sex[['person_id','Sex']]
#         data = pd.merge(data, sex, how='left', on=['person_id'])
        #data=data[data['Age']<=10]
        data=data.sort_values(by=['person_id','Age'])
        #print(data['RANDOM_PATIENT_ID'].unique())
        del data['person_id']
        del data['visit_start_datetime']
        del data['visit_end_datetime']
        del data['val']
        del data['Interval']
        #print(data.shape)
        #print(data.columns)
        data.rename({'MEAS_3038553': 'BMI'}, axis=1, inplace=True)
        
        if(self.flag==0):
            data=data.replace(np.inf,-1)
            data=data.replace(-1,np.nan)
            
            #print(self.flag)
            cols_to_norm = [col for col in data if (col.startswith('MEAS_') or col.startswith('C_MEAS_') or col.startswith('Weight'))]
            data_norm=StandardScaler()
            #bmi_scaler.fit(bmi)
            data[cols_to_norm]=data_norm.fit_transform(data[cols_to_norm])

            
        
            data=data.replace(np.nan,0)

           # data = data.to_numpy()
            #data = np.reshape(data, (int(data.shape[0]/99), 99, data.shape[1]))
            data = T.as_tensor(data.values.astype(float), dtype=T.float32)
            #data=T.from_numpy(data)
            #data=data.double()
            data=data.view(int(data.shape[0]/self.seq_len), self.seq_len, data.shape[1])
            return data
        else:
            #print(self.flag)
            data=data.replace(np.nan,0)
            data = T.as_tensor(data.values.astype(float), dtype=T.float32)
            data=data.view(int(data.shape[0]/self.seq_len), self.seq_len, data.shape[1])
            return data

    def __len__(self):
        return self.len

In [5]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf#11.1179
        self.delta = delta

    def __call__(self, val_loss, model, optimizer, save_path):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, optimizer, save_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, optimizer, save_path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, optimizer, save_path):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        T.save({
            "model": model.state_dict(),
            'trainer': optimizer.state_dict()
        }, save_path)
        self.val_loss_min = val_loss

In [6]:
def pred_test(args, model,predWin):
    model.eval()
    
    RLoss=0
    FLoss=0
    mseLoss=0
    mseLossF=0
    TBatches=0
    oBmi=[]
    oBmiF=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    imputations=[]
    with T.autograd.no_grad():
        files = '.../aaai/data/ehr/preprocess/test/finalTest1054.csv'
    
        maskFiles = '.../aaai/data/ehr/preprocess/test/mask/maskTest1054.csv'
        
        dataset = CSVDataset(files, int(args.seq_len*500),1356100,args.seq_len,flag=0)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*500),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            #bmi_norm=dataset.bmi_norm
            #print('batch: {}'.format(batch_idx))
            data,mask=allData
            decay=mask[:,:,:,608]
            rdecay=mask[:,:,:,609]
            mask=mask[:,:,:,316]

            data=data.squeeze()
            mask=mask.squeeze()
            decay=decay.squeeze()
            rdecay=rdecay.squeeze()
            #print(data.shape)
            #print(mask.shape)
            #print(decay.shape)
            
            #values to be predicted
            y = data.clone().detach()
            testMask = mask.clone().detach()
#             sex=y[:,:,608]
#             age=y[:,:,607]
            #print(sex.shape,age.shape)
            y=y[:,:,316]
            #print(y[0])
            data=data[:,:,0:608]
            
            #------------remove last 5 timestamps------------------
            #print(data[0:10,8:,653])
            for i in range(data.shape[0]):
                #if(data[i,])
                j=20
                if(predWin==8):
                    k=16
                    decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8])
                    rdecay[i,j-k:j]=T.tensor([8,7.5,7,6.5,6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])
                elif(predWin==7):
                    k=14
                    decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7])
                    rdecay[i,j-k:j]=T.tensor([7,6.5,6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])*120
                elif(predWin==6):
                    k=12
                    decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6])
                    rdecay[i,j-k:j]=T.tensor([6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])
                elif(predWin==5):
                    k=10
                    decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5])
                    rdecay[i,j-k:j]=T.tensor([5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])
                
                data[i,j-k:j,:]=0
                mask[i,j-k:j]=0
                y[i,0:j-k]=0
#                 age[i,0:j-k]=0
#                 sex[i,0:j-k]=0
                #print(sex.shape,age.shape)
                #yOrig[i,0:j-k]=0
                testMask[i,0:j-k]=0


            ret_f, ret = run_on_batch(model,data,mask,decay,rdecay,args, optimizer=None)#,bmi_norm)
#             print("Input",data.shape)
#             print(data[0,:,316])
#             print("Reverse",ret['imputations'][0,:])
#             print("ForwardOnly",ret_f['imputations'][0,:])
#             print("Original",y.shape)
#             print(y[0,:])
#             print("Mask",testMask.shape)
#             print(testMask[0,:])
            RLoss=RLoss+ret['loss']
            FLoss=FLoss+ret_f['loss']
            testMask=testMask.cuda()
            y=y.cuda()
            outputBMI=ret['imputations'] * testMask
            outputBMIF=ret_f['imputations'] * testMask
            mseLoss=mseLoss+ (torch.sum(torch.abs(outputBMI-y)))/ (torch.sum(testMask) + 1e-5)
            mseLossF=mseLossF+ (torch.sum(torch.abs(outputBMIF-y)))/ (torch.sum(testMask) + 1e-5)
#             print("RMSELoss Revrese: ",mseLoss)
#             print("RMSELoss Forward: ",mseLossF)
            outBmi, outBmiF,inBmi = plotBmi(outputBMI, outputBMIF, y, testMask)
            oBmi.extend(outBmi)
            oBmiF.extend(outBmiF)
            iBmi.extend(inBmi)

            #T.cuda.empty_cache()
            #paramsE=list(model['e'].parameters())
            #paramsG=list(model['g'].parameters())
            #print("AFTER PARAM",paramsE[0][20],paramsG[8][0][0])  
        TBatches=TBatches+batch_idx+1
        RLoss = RLoss/TBatches
        mseLoss = mseLoss/TBatches
        mseLossF = mseLossF/TBatches
    #print("===================================")
    oBmi=np.asarray(oBmi)
    iBmi=np.asarray(iBmi)
    loss = (oBmi - iBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mseLoss) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))

    #print("Val R Loss:",RLoss)
    print("CI",ci)
    print("MAE Loss Reverse:",mseLoss)
    #print("MAE Loss Forward:",mseLossF)
    #print(outputBMI)
    return oBmi,oBmiF,iBmi
                
                

In [7]:
def imputation_test(args, model,missingRate):
    model.eval()
    
    RLoss=0
    FLoss=0
    mseLoss=0
    mseLossF=0
    TBatches=0
    oBmi=[]
    oBmiF=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    imputations=[]
    samples=0
    pids=0
    with T.autograd.no_grad():
        files = '.../aaai/data/ehr/preprocess/test/finalTest1054.csv'
    
        maskFiles = '.../aaai/data/ehr/preprocess/test/mask/maskTest1054.csv'
        
        dataset = CSVDataset(files, int(args.seq_len*500),1356100,args.seq_len,flag=0)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*500),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            #bmi_norm=dataset.bmi_norm
            #print('batch: {}'.format(batch_idx))
            data,mask=allData
            decay=mask[:,:,:,608]
            rdecay=mask[:,:,:,609]
            mask=mask[:,:,:,316]

            data=data.squeeze()
            mask=mask.squeeze()
            decay=decay.squeeze()
            rdecay=rdecay.squeeze()
            #print(data.shape)
            #print(mask.shape)
            #print(decay.shape)
            
            #values to be predicted
            y = data.clone().detach()
            testMask = mask.clone().detach()
#             sex=y[:,:,608]
#             age=y[:,:,607]
            #print(sex.shape,age.shape)
            y=y[:,:,316]
            #print(y[0])
            data=data[:,:,0:608]
            
            #------------remove last 5 timestamps------------------
            #print(data[0:10,8:,653])
            for i in range(data.shape[0]):
                #if(data[i,])
                j=20
                k=16
                #mask[i,:].loc[mask[i,:].query('value == 1').sample(frac=.1).index,'value'] = 0
                idxs = torch.nonzero(mask[i,:] == 1)
                samples=samples+list(idxs.size())[0]
                if(missingRate==50):
                    if(list(idxs.size())[0]>4):
                        idxs=random.sample(set(idxs),5)
                        data[i,idxs[0],316]=0
                        data[i,idxs[1],316]=0
                        data[i,idxs[2],316]=0
                        data[i,idxs[3],316]=0
                        data[i,idxs[4],316]=0
                        #print(mask[i,:])
                        mask[i,idxs[0]]=0
                        mask[i,idxs[1]]=0
                        mask[i,idxs[2]]=0
                        mask[i,idxs[3]]=0
                        mask[i,idxs[4]]=0
                        pids=pids + 5
                        break;
                if(missingRate>=40):
                    if(list(idxs.size())[0]>3):
                        idxs=random.sample(set(idxs),4)
                        data[i,idxs[0],316]=0
                        data[i,idxs[1],316]=0
                        data[i,idxs[2],316]=0
                        data[i,idxs[3],316]=0
                        #print(mask[i,:])
                        mask[i,idxs[0]]=0
                        mask[i,idxs[1]]=0
                        mask[i,idxs[2]]=0
                        mask[i,idxs[3]]=0
                        pids=pids + 4
                        break;
                if(missingRate>=30):
                    if(list(idxs.size())[0]>2):
                        idxs=random.sample(set(idxs),3)
                        data[i,idxs[0],316]=0
                        data[i,idxs[1],316]=0
                        data[i,idxs[2],316]=0
                        #print(mask[i,:])
                        mask[i,idxs[0]]=0
                        mask[i,idxs[1]]=0
                        mask[i,idxs[2]]=0
                        pids=pids + 3
                        break;
                if(missingRate>=20):
                    if(list(idxs.size())[0]>1):
                        idxs=random.sample(set(idxs),2)
                        data[i,idxs[0],316]=0
                        data[i,idxs[1],316]=0
                        #print(mask[i,:])
                        mask[i,idxs[0]]=0
                        mask[i,idxs[1]]=0
                        pids=pids + 2
                        break;
                if(missingRate>=10):
                    if(list(idxs.size())[0]>0):
                        idxs=random.sample(set(idxs),1)
                        data[i,idxs,316]=0
                        mask[i,idxs]=0
                        pids=pids + 1

                #remove values from last 50% of the sequence
                
                #print(mask[i,:])
                testMask[i,:]=testMask[i,:]-mask[i,:]
                #print(testMask[i,:])
                #print(y[i,:])
                y[i,:]=y[i,:]*testMask[i,:]
#                 age[i,:]=age[i,:]*testMask[i,:]
#                 sex[i,:]=sex[i,:]*testMask[i,:]
                

            ret_f,ret = run_on_batch(model,data,mask,decay,rdecay,args, optimizer=None)#,bmi_norm)
#             print("Input",data.shape)
#             print(data[0,:,316])
#             print("Reverse",ret['imputations'][0,:])
#             print("ForwardOnly",ret_f['imputations'][0,:])
#             print("Original",y.shape)
#             print(y[0,:])
#             print("Mask",testMask.shape)
#             print(testMask[0,:])
            RLoss=RLoss+ret['loss']
            FLoss=FLoss+ret_f['loss']
            testMask=testMask.cuda()
            y=y.cuda()
            outputBMI=ret['imputations'] * testMask
            outputBMIF=ret_f['imputations'] * testMask
            mseLoss=mseLoss+ (torch.sum(torch.abs(outputBMI-y)))/ (torch.sum(testMask) + 1e-5)
            mseLossF=mseLossF+ (torch.sum(torch.abs(outputBMIF-y)))/ (torch.sum(testMask) + 1e-5)
#             print("RMSELoss Revrese: ",mseLoss)
#             print("RMSELoss Forward: ",mseLossF)
            outBmi, outBmiF,inBmi = plotBmi(outputBMI, outputBMIF, y, testMask)
            oBmi.extend(outBmi)
            oBmiF.extend(outBmiF)
            iBmi.extend(inBmi)

        TBatches=TBatches+batch_idx+1
        RLoss = RLoss/TBatches
        mseLoss = mseLoss/TBatches
        mseLossF = mseLossF/TBatches
    #print("===================================")
    oBmi=np.asarray(oBmi)
    iBmi=np.asarray(iBmi)
    loss = (oBmi - iBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mseLoss) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))

    #print("Val R Loss:",RLoss)
    print("CI",ci)
    print("MAE Loss Reverse:",mseLoss)
    #print("MAE Loss Forward:",mseLossF)
    #print(outputBMI)
    return oBmi,oBmiF,iBmi
                
                

In [8]:
def plotBmi(outBmi,outBmiF, inBmi, testMask):
    
    outBmi = outBmi.cpu().detach().numpy()
    outBmiF = outBmiF.cpu().detach().numpy()
    inBmi = inBmi.cpu().detach().numpy()
    testMask = testMask.cpu().detach().numpy()
    
    #import matplotlib.pyplot as plt
    #%matplotlib inline
    #from matplotlib.ticker import MultipleLocator
    outBmi=outBmi[np.nonzero(testMask)]
    outBmiF=outBmiF[np.nonzero(testMask)]
    inBmi=inBmi[np.nonzero(testMask)]
    
    #print(outBmi)
    #print(inBmi)
    #print(outAge)
    #print(outSex)
    
    #print(sex.shape,age.shape)
    
    return outBmi,outBmiF,inBmi

In [9]:
def run_evalFull(args, model):
    model.eval()
    
    RLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oSex=[]
   
    with T.autograd.no_grad():
        files = '.../aaai/data/ehr/preprocess/val/finalVal1054.csv'
    
        maskFiles = '.../aaai/data/ehr/preprocess/val/mask/maskVal1054.csv'
        #print(files)
        
        dataset = CSVDataset(files, int(args.seq_len*500),1356100,args.seq_len,flag=0)
        #orig = CSVDataset('C:\\Users/mehak/Desktop/testganAggOrig.csv', int(args.seq_len*500),1356100,args.seq_len)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*500),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        #origLoader = DataLoader(orig,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            #bmi_norm=dataset.bmi_norm
            #print('batch: {}'.format(batch_idx))
            data,mask=allData
            decay=mask[:,:,:,608]
            rdecay=mask[:,:,:,609]
            mask=mask[:,:,:,316]

            data=data.squeeze()
            mask=mask.squeeze()
            decay=decay.squeeze()
            rdecay=rdecay.squeeze()
            #print(data.shape)
            #print(mask.shape)
            #print(decay.shape)


            ret_f, ret = run_on_batch(model,data,mask,decay,rdecay, args, optimizer=None)#,bmi_norm)
            RLoss=RLoss+ret['loss']

            #T.cuda.empty_cache()
            #paramsE=list(model['e'].parameters())
            #paramsG=list(model['g'].parameters())
            #print("AFTER PARAM",paramsE[0][20],paramsG[8][0][0])  
        TBatches=TBatches+batch_idx+1
        RLoss = RLoss/TBatches
    #print("===================================")
    print("Val R Loss:",RLoss)
    #print(outputBMI)
    return RLoss
                
                

In [10]:
def run_epoch(args, model):
    ''' Run a single epoch
    '''

    trainLoss=[]
    valLoss=[]
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    if args.resume_training:
        early_stopping(133, model, optimizer, save_path)
        
    #define optimizer
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    
    #for evrey epoch
    for epoch in range(args.num_epochs):
        model.train()
    
        #Running Losses
        RLoss=0
        TBatches=0  
        print("=============EPOCH=================")
       
        path = '.../aaai/data/ehr/preprocess/train/'
        files = list(map(lambda x : path + x, (filter(lambda x : x.endswith("csv"), os.listdir(path)))))
        
        maskPath = '.../aaai/data/ehr/preprocess/train/mask/'
        maskFiles = list(map(lambda x : maskPath + x, (filter(lambda x : x.endswith("csv"), os.listdir(maskPath)))))
        
        #print(files)
        #print(maskFiles)
        #print("====================New File========================")
        dataset = CSVDataset(files[0], int(args.seq_len*500),1356100,args.seq_len,flag=0)
        maskDataset = CSVDataset(maskFiles[0], int(args.seq_len*500),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, maskLoader)):
            #bmi_norm=dataset.bmi_norm
            #print('batch: {}'.format(batch_idx))
            data,mask=allData
            decay=mask[:,:,:,608]
            rdecay=mask[:,:,:,609]
            mask=mask[:,:,:,316]

            data=data.squeeze()
            mask=mask.squeeze()
            decay=decay.squeeze()
            rdecay=rdecay.squeeze()
            #print(data.shape)
            #print(mask.shape)
            #print(decay.shape)


            ret_f, ret = run_on_batch(model,data,mask,decay,rdecay, args, optimizer)#,bmi_norm)
            RLoss=RLoss+ret['loss'].item()

            #T.cuda.empty_cache()
            #paramsE=list(model['e'].parameters())
            #paramsG=list(model['g'].parameters())
            #print("AFTER PARAM",paramsE[0][20],paramsG[8][0][0])   

        TBatches=TBatches+batch_idx+1
        #print(TBatches)
        #print("File:", i, "loss_R:", "%.4f"%RLoss/(batch_idx+1), "loss_G:", "%.4f"%GLoss/(batch_idx+1), "loss_D:", "%.4f"%DLoss/(batch_idx+1))
            #print(len(encoded))
        RLoss=RLoss/TBatches
        
        print("EPOCH:", epoch, "loss_R:", "%.4f"%RLoss)
        
        trainLoss.append(RLoss)

        valid_loss = run_evalFull(args, model)
        
        valLoss.append(valid_loss)
        #plotBmi(outBmi , inBmi)

        
        #if epoch<1 or epoch >5:
        if not (T.isnan(valid_loss)):
            early_stopping(valid_loss, model, optimizer, save_path)

        if early_stopping.early_stop:
            print("Early stopping")
            break

        #plot_grad_flow(model['e'].named_parameters())
        #plot_grad_flow(model['g'].named_parameters())
        #plot_grad_flow(model['d'].named_parameters())
    return trainLoss, valLoss


In [11]:
def run(args):
    
    train_on_gpu = T.cuda.is_available()
    if train_on_gpu:
        print('Training on GPU.')
    else:
        print('No GPU available, training on CPU.')
        
    model = BRITS2()
    
    #print("Model",model)

    if torch.cuda.is_available():
        model = model.cuda()

    if args.resume_training:
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint)
        optimizer.load_state_dict(checkpoint)
        output = run_epoch(args, model) 
        #return model,output
    
    elif args.train:
        trainLoss, valLoss = run_epoch(args, model) 
        #return trainLoss, valLoss
        
    elif args.evalImp:
        #load Model
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint['model'])
        optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        optimizer.load_state_dict(checkpoint['trainer'])
        oBmi, oBmiF, iBmi = imputation_test(args, model,args.missingRate)
        
    elif args.evalPred:
        #load Model
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint['model'])
        optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        optimizer.load_state_dict(checkpoint['trainer'])
        oBmi, oBmiF, iBmi = pred_test(args, model,args.pred_len)
        
        #return oBmi, oBmiF, iBmi

In [12]:
#trainLoss, valLoss = run(ARGS)
#oBmi, oBmiF, iBmi, oAge, oSex = run(ARGS)
run(ARGS)

Training on GPU.
MSE Loss Reverse: tensor(2.8642, device='cuda:0')
Missing%:  0.21005759069049457
